In [ ]:
from marvin.tools.query import Query
from marvin.tools.maps import Maps
import numpy as np

In [ ]:
tot = [] #All galaxies
agns = [] #AGNs

In [ ]:
def classify_gal(gal_id):
    """Uses BPT and WHAN criteria on the center spaxel to determine
    whether or not the galaxy hosts an AGN

    Parameters:
    gal_id - MaNGA id of the galaxy

    Returns:
    Function has no return value, however, appends galaxies that
    fulfill the criteria to the list of AGNs
    """
    maps = Maps(mangaid=gal_id) 
    center = int(len(maps.emline_gflux_ha_6564)/2)
    
    ha = maps.emline_gflux_ha_6564[center][center]
    nii = maps.emline_gflux_nii_6585[center][center]
    hb = maps.emline_gflux_hb_4862[center][center]
    oiii = maps.emline_gflux_oiii_5008[center][center]
    oi = maps.emline_gflux_oi_6302[center][center]
    
    sii_6718 = maps.emline_gflux_sii_6718[center][center]
    sii_6732 = maps.emline_gflux_sii_6732[center][center]
    
    log_nii_ha = np.log10(nii.value/ha.value)
    log_oiii_hb = np.log10(oiii.value/hb.value)
    log_sii_ha = np.log10((sii_6718.value + sii_6732.value)/ha.value)
    log_oi_ha = np.log10(oi.value/ha.value)
    
    bpt_crit_1 = (0.61/(log_nii_ha - 0.47) + 1.19 < log_oiii_hb)
    bpt_crit_2 = (0.72/(log_sii_ha - 0.32) + 1.30 < log_oiii_hb)
    bpt_crit_3 = (0.73/(log_oi_ha + 0.59) + 1.33 < log_oiii_hb)
    
    if bpt_crit_1 and bpt_crit_2 and bpt_crit_3:
        
        ewha = maps.emline_sew_ha_6564[center][center].value
        ewnii = maps.emline_sew_nii_6585[center][center].value
        
        if log_nii_ha > -0.4 and ewha > 3:
            agns.append(gal_id)
    

In [ ]:
q = Query(limit=10000)
r = q.run()

for i in range(r.totalcount):
    tot.append(r.results[i].mangaid)
    classify_gal(r.results[i].mangaid)

In [ ]:
print(len(tot))

In [ ]:
print(tot)

In [ ]:
print(len(agns))

In [ ]:
print(agns)

In [ ]:
with open('AGN MaNGAids.txt', 'w') as f:
    for gal_id in agns:
        f.write("%s\n" % gal_id)